In [1]:
import os
os.chdir('..')

In [2]:
import numpy as np
import scipy as sp
import datajoint as dj
import matplotlib.pyplot as plt

In [5]:
dj.config.load('dj_local_conf_template.json')

In [6]:
from pipeline import lab, experiment, tracking, ephys, psth
from pipeline import plot
from pipeline.plot import behavior_plot, unit_characteristic_plot

Connecting thinh@mesoscale-activity.datajoint.io:3306


# Top level ***ephys*** diagram

In [ ]:
ephys_erd = ((dj.ERD(ephys.Unit)-2) + dj.ERD(ephys.TrialSpikes)
             + (dj.ERD(experiment.BrainLocation)-1)
             + dj.ERD(ephys.ProbeInsertion.InsertionLocation)
             + dj.ERD(psth.UnitSelectivity) + (dj.ERD(psth.PeriodSelectivity) - 1))

In [ ]:
ephys_erd

# Top level ***behavior*** diagram

In [ ]:
behav_erd = (dj.ERD(experiment.SessionTrial) - 1 
             + (dj.ERD(experiment.BrainLocation)-1) 
             + (dj.ERD(experiment.BehaviorTrial) + 1 - 1) 
             + (dj.ERD(experiment.PhotostimTrial) + 1 - 1)) 

In [ ]:
behav_erd

# Browsing of subjects

In [ ]:
lab.Subject()

### Query one subject

In [ ]:
# restrict by string
lab.Subject & 'subject_id=397853'

In [ ]:
# restrict by dictionary
lab.Subject & {'subject_id': 397853}

In [ ]:
lab.Subject & {'sex': 'm'}

### Fetch back the subject key

In [ ]:
subj_397853 = (lab.Subject & {'subject_id': 397853}).fetch1('KEY')  # "fetch1()" because we know there's only one

In [ ]:
subj_397853

In [ ]:
male_subjs = (lab.Subject & {'sex': 'm'}).fetch('KEY')  # "fetch()" for fetching multiple

In [ ]:
male_subjs

# Browsing of sessions

In [ ]:
# Get sessions with units
sessions = experiment.Session & ephys.Unit
sessions

In [ ]:
# List all sessions and the ProbeInsertion counts for each
sessions = sessions.aggr(ephys.ProbeInsertion, probe_insert_count='count(*)')
sessions

In [ ]:
# query 1 session
experiment.Session & {'session': 15, 'subject_id': 412330}

In [ ]:
# get session key
session_key = (experiment.Session & {'session': 1, 'subject_id': 435884}).fetch1('KEY')

# Browsing of units for this session
#### Keep in mind that units may come from different ***ProbeInsertion***

In [ ]:
unit_erd = (dj.ERD(ephys.Unit) - 1) + (dj.ERD(ephys.Unit) + 1)

In [ ]:
unit_erd

In [ ]:
ephys.Unit & session_key

In [ ]:
good_units = ephys.Unit & session_key & {'unit_quality': 'good'}
good_units

## Unit selectivity

In [ ]:
psth.UnitSelectivity & good_units

### Pick one unit

In [ ]:
unit_key = (psth.UnitSelectivity & good_units & 'unit=5').fetch1('KEY')
unit_key                                                                 

### Unit PSTH

In [ ]:
upsth_erd = (dj.ERD(psth.UnitPsth) - 1)

In [ ]:
upsth_erd

In [ ]:
psth.TrialCondition()

In [ ]:
unit_psth = psth.UnitPsth * psth.TrialCondition & unit_key & 'trial_condition_desc = "good_noearlylick_hit"'

In [ ]:
psth, edges = unit_psth.fetch1('unit_psth')

In [ ]:
plt.plot(edges[1:], psth)

In [ ]:
plot.unit_psth(unit_key)

# ================================================

In [ ]:
session_key={'session_id': 15, 'subject_id': 412330, 'unit_quality': 'good'}
period_selectivity_key_source = experiment.Period * ephys.Unit & session_key

In [ ]:
experiment.BrainLocation()

In [ ]:
ephys.ProbeInsertion()

In [ ]:
probe_insertions = {'session_id': 15, 'subject_id': 412330, 'insertion_number': 1}

In [ ]:
probe_insertions = {'session_id': 15, 'subject_id': 412330, 'insertion_number': 1}
ephys.ProbeInsertion.InsertionLocation.insert1({**probe_insertions, 'brain_location_name': 'right_alm'})
probe_insertions = {'session_id': 15, 'subject_id': 412330, 'insertion_number': 2}
ephys.ProbeInsertion.InsertionLocation.insert1({**probe_insertions, 'brain_location_name': 'right_medulla'})

In [ ]:
probe_insertions = {'session_id': 15, 'subject_id': 412330, 'insertion_number': 2}
ephys.ProbeInsertion.InsertionLocation({**probe_insertions, 'brain_location_name': 'right_medulla'})

In [ ]:
ephys.ProbeInsertion.InsertionLocation.heading

In [ ]:
experiment.Session.heading

In [7]:
insertion_key2 = (ephys.ProbeInsertion & {'session_id': 15, 'subject_id': 412330}).fetch('KEY')[1]

subject_id institution 6 digit animal ID,session session number,insertion_number,probe unique identifier of this probe (e.g. serial number),electrode_config_id hash of the group and group_member (ensure uniqueness)


In [9]:
ephys.ProbeInsertion()

subject_id institution 6 digit animal ID,session session number,insertion_number,probe unique identifier of this probe (e.g. serial number),electrode_config_id hash of the group and group_member (ensure uniqueness)
